In [1]:
import os
import torch
import torchaudio
from matplotlib import pyplot as plt
from IPython.display import Audio, display
import math
import numpy as np
import pandas as pd

In [7]:
class Station:
    
    def __init__(self, name):
        self.name = name
        self.events = []
        
    def set_event(self, event):
        self.events.append(event)

class Event:
    
    def __init__(self, name, wav_dir):
        self.name = name
        self.station = ''
        self.kind = ''
        self.duration = 0.0
        self.power = 0.0
        self.calculator(name, wav_dir)
    
    def calculator(self, audio, wav_dir):
        self.kind_set(audio)
        self.audio_average_power(wav_dir)
    
    def audio_average_power(self, wav_dir):
         
#         main = '/home/kasparov/Documentos/20190114_vpoblete/20221019_Wasserstein/split_signals_into_labels_and_noise_20221020_for_ronda1/'
        main = "Z:/Proyectos/AuMiLab/FUSA/Noise/fusa_power/"

        wav_path = os.path.join(main + wav_dir, self.name)
        waveform, sample_rate = torchaudio.load(wav_path)
        num_channels, num_frames = waveform.shape
        
        if waveform.numpy().shape[1] != 0:
            
            ft_wf = np.fft.fft(waveform.numpy())/(math.sqrt(waveform.numpy().shape[1]))
            
            parseval_waveform = np.sum(waveform.numpy()**2)
            parseval_fourier_transform = np.sum(np.abs(ft_wf)**2)
            
            if (np.float32(round(parseval_waveform,4)) == np.float32(round(parseval_fourier_transform,4))):
                
                self.power = parseval_waveform
                
                duration = torch.arange(0, num_frames) / sample_rate
                self.duration = duration.numpy()[-1]
            
    def kind_set(self, audio):
        text = audio.split('_')
        self.kind = text[2]

def folders(wav_dir, station):
    
    noise = 'ruido/'
    not_noise = 'audios_all_classes/'
    wav_noise_names = os.listdir(wav_dir + noise)
    wav_not_noise_names = os.listdir(wav_dir + not_noise)
    
    for w in wav_noise_names:
        station.set_event(Event(w, wav_dir + noise))
    
    for nn in wav_not_noise_names:
        station.set_event(Event(nn, wav_dir + not_noise))
    
    return station

def dataframer(stations):
    
    thrshold = 0.5
    data = []
    framer = []
    
    for s in stations:
        
        data.clear()
        
        for e in s.events:
            
            if e.duration >= thrshold:
                data.append([s.name, e.name, e.kind, e.duration, e.power])
        
        framer.append(pd.DataFrame(data.copy(), columns = ['station', 'audio', 'class', 'duration', 'power']))
    
    return framer

#returns a slice of dataframes!!
def _initiator():
    stations = []
    fls = []

    main = "Z:/Proyectos/AuMiLab/FUSA/Noise/fusa_power/"
#     wav_dir = '/home/kasparov/Documentos/20190114_vpoblete/20221019_Wasserstein/split_signals_into_labels_and_noise_20221020_for_ronda1/'
    
    fls.append("split_signals_into_labels_and_noise_20221020_for_ronda1/")
    fls.append("split_signals_into_labels_and_noise_20221022_for_ronda2/")
    fls.append("split_signals_into_labels_and_noise_20221020_for_ronda3/")
    fls.append("split_signals_into_labels_and_noise_20221020_for_ronda4/")
    fls.append("split_signals_into_labels_and_noise_20221020_for_ronda5/")
    fls.append("split_signals_into_labels_and_noise_20221022_for_ronda6/")
    
    
    for f in fls:
        stations.append(Station(f.split("_")[8]))
    
    for s in stations:
        
        for f in fls:
            if(f.split("_")[8] == s.name):
                s = folders(f, s)
                break
            
    return stations

In [3]:
stations = _initiator()

In [8]:
df = dataframer(stations)

In [10]:
df[1]

,station,audio,class,duration,power
0,ronda2/,1655671051_1655671111_ruido_60.wav,ruido,28.344999,8.212375
1,ronda2/,1655721332_1655721392_Viento_104.wav,Viento,31.520000,1.810563


In [13]:
df=df[1].loc[df[1]['class']=='ruido']

In [130]:
df

,station,audio,class,duration,power
0,Capta002,1653712794_1653712854_ruido_223.wav,ruido,13.647708,0.754834
1,Capta002,1653789034_1653789094_ruido_417.wav,ruido,2.769979,0.419892
2,Capta002,1653706209_1653706269_ruido_160.wav,ruido,27.045750,1.402200
3,Capta002,1655255968_1655256028_ruido_1248.wav,ruido,4.182500,0.074135
4,Capta002,1655063894_1655063954_ruido_1159.wav,ruido,1.474979,0.218602
...,...,...,...,...,...
1241,Capta002,1655234087_1655234147_Camion_movimiento_1188.wav,Camion,8.582750,10.789660
1242,Capta002,1653700210_1653700270_Viento_71.wav,Viento,7.430313,1.868526
1243,Capta002,1655236693_1655236753_Ave_1239.wav,Ave,1.444854,0.717275
1244,Capta002,1654985801_1654985861_Perro_1044.wav,Perro,1.899979,33.140285


In [133]:
df=df.loc[df['class']=='ruido']

In [134]:
df.sort_values(by=['power'])

,station,audio,class,duration,power
151,Capta002,1655255968_1655256028_ruido_1254.wav,ruido,0.532313,0.008243
29,Capta002,1653981393_16539814531_ruido_586.wav,ruido,0.519958,0.009991
74,Capta002,1653981393_16539814531_ruido_598.wav,ruido,0.699979,0.011472
152,Capta002,1653981393_16539814531_ruido_576.wav,ruido,0.662229,0.012990
24,Capta002,1653787701_1653787761_ruido_376.wav,ruido,0.639979,0.016448
...,...,...,...,...,...
168,Capta002,1653680026_1653680086_ruido_28.wav,ruido,7.988875,20.741306
193,Capta002,1654270899_16542709591_ruido_867.wav,ruido,3.693292,23.779062
115,Capta002,1654270899_16542709591_ruido_870.wav,ruido,1.666646,30.190453
211,Capta002,1654270899_16542709591_ruido_864.wav,ruido,4.486646,52.609501


In [137]:
df2=df.loc[df['class']!='ruido']

In [138]:
df2.sort_values(by=['power'])

,station,audio,class,duration,power


In [119]:
station.events[800].duration

1.5499792

In [79]:
na = e.duration.numpy()
print(na[-1])

59.99998


In [29]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [136]:
df.to_csv('power_out.csv', index=False)

In [ ]:
def audio_average_power(wav_dir, wav_names):
    
    average_power_time = []
    average_power_ft_wf = []
    for j in range(len(wav_names)):
        
        wav_path = os.path.join(wav_dir, wav_names[j])
        waveform, sample_rate = torchaudio.load(wav_path)
        
        if waveform.numpy().shape[1] != 0:
            
            ft_wf = np.fft.fft(waveform.numpy())/(math.sqrt(waveform.numpy().shape[1]))
            parseval_waveform = np.sum(waveform.numpy()**2)
            average_power_time.append(parseval_waveform)
            
            parseval_fourier_transform = np.sum(np.abs(ft_wf)**2)
            average_power_ft_wf.append(parseval_fourier_transform)
            #print(average_power_time, " equals ", average_power_ft_wf)
        #average_power_time = [item for item in average_power_time if item <= 100]
        #average_power_ft_wf = [item for item in average_power_ft_wf if item <= 100]
        
    return average_power_time, average_power_ft_wf

def plot_waveform(waveform, sample_rate, title="power", xlim=None, ylim=None):
    waveform = waveform.numpy()
    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].plot(time_axis, waveform[c], linewidth=1)
        axes[c].grid(True)
        if num_channels > 1:
            axes[c].set_ylabel(f'Channel {c+1}')
        if xlim:
            axes[c].set_xlim(xlim)
        if ylim:
            axes[c].set_ylim(ylim)
    figure.suptitle(title)
    plt.show(block=False)

def play_audio(waveform, sample_rate):
    waveform = waveform.numpy()
    
    num_channels, num_frames = waveform.shape
    
    if num_channels == 1:
        display(Audio(waveform[0], rate=sample_rate))
    elif num_channels == 2:
        display(Audio((waveform[0], waveform[1]), rate=sample_rate))
    else:
        raise ValueError("Waveform with more than 2 channels are not supported.")

In [ ]:
classes = 'audios_all_classes/'
ruido   = 'ruido/'
actua_round = 'ronda1/'
noise_path  = '/home/kasparov/Documentos/20190114_vpoblete/20221019_Wasserstein/split_signals_into_labels_and_noise_20221020_for_' + actua_round + ruido

wav_noise_names = os.listdir(noise_path)


In [ ]:
# posición
for a in range(len(wav_noise_names)):
    if '1653683601_1653683661_ruido_61.wav'==wav_noise_names[a]:
        print(a)
        break

In [ ]:
w = wav_noise_names.index('1653712855_1653712915_ruido_231.wav') 

In [ ]:
print(wav_noise_names[233])

In [ ]:
waveform, sample_rate = torchaudio.load(noise_path + wav_noise_names[w])

In [ ]:
plot_waveform(waveform, sample_rate)

In [ ]:
play_audio(waveform, sample_rate)